# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM이 외부 데이터를 컨텍스트로 활용.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding.embed_query('점심 배불리 먹었더니 졸리다..')
# print(len(x), x)

# VectorStore 에 임베딩 후 저장(In memory)
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
)

In [ ]:
vectorstore.similarity_search(
    '초보자가 키우기 좋은 애완동물 추천해줘', k=2
)

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [ ]:
%pip install -q pymupdf

In [24]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()


print('원본 pdf의 장수', len(docs))

원본 pdf의 장수 23


In [25]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [26]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='bca8a6e2-6d02-4b67-b71f-bd049da74cf7', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

## 검색 증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM 
4. 최종 답변

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅 (랭체인 허브에서 가져오기)
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

'삼성전자는 자체 개발한 생성 AI ‘삼성 가우스’를 2023년 11월 공개했으며, 언어, 코드, 이미지 모델로 구성되어 온디바이스에서 작동 가능하게 설계되었습니다. 삼성 가우스는 사용자 정보 유출 위험이 없으며, 단계적으로 다양한 제품에 탑재될 예정입니다. 이 AI는 메일 작성, 문서 요약, 이미지 생성 등 여러 업무를 지원하며, 경쟁 제품과의 차별성을 갖추고 있습니다.'

In [39]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""


prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)

In [41]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '삼성전자 AI 최신동향 2025년'}`


{'query': '삼성전자 AI 최신동향 2025년', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://blog.naver.com/kcc_press/223765970551?fromRss=true&trackingCode=rss', 'title': '2025년 2-1호(제53호) [이슈 트렌드] 온디바이스 AI의 부상과 ...', 'content': "삼성전자는 최신 모바일 애플리케이션 프로세서(AP) '엑시노스(Exynos) 2400'을 통해 기존 대비 NPU 성능을 14.7배 향상시키며 AI 연산 성능을 대폭 강화", 'score': 0.84918517, 'raw_content': None}, {'url': 'https://voiceinvest.tistory.com/161', 'title': '2025 삼성전자 분석: 혁신 기술과 시장 전략으로 선도하는 미래!', 'content': '삼성전자는 글로벌 전자산업의 선두주자로 자리잡으며, 지속 가능한 발전과 고객 중심의 혁신을 통해 미래를 이끌어가고 있습니다. 삼성전자는 아시아 시장에서의 성장을 통해 브랜드의 지배력을 더욱 확고히 하고 있습니다. 특히 반도체 산업과 인공지능(AI), 사물인터넷(IoT) 분야에서의 기술 발전은 삼성전자의 경쟁력을 더욱 강화하고 있습니다. 삼성전자는 반도체 기술 분야에서 지속적인 혁신을 통해 글로벌 시장에서 경쟁 우위를 확보하고 있습니다. 1.   **스마트 홈 기술**: 삼성전자는 IoT 기반의 스마트 홈 기술을 통해 사용자 경험을 혁신하고 있습니다. AI 및 IoT 분야에서의 성장은 만연한 추세이며, 삼성전자의 높은 R&D 투자와 기술 개발이 이러한 미래 기술을 선도하는 데 중요한

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='pdf 기준 삼성전자 소식들 요약해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='삼성전자는 최근 자체 개발한 생성형 AI인 ‘삼성 가우스’를 공개하였으며, 이 AI는 온디바이스에서 작동 가능하며 언어, 코드, 이미지 모델로 구성되어 있습니다. 삼성 가우스는 사용자 정보 유출 위험이 없고, 텍스트 생성, 코드 작성, 이미지 창작 등 다양한 작업을 지원합니다. 또한, 삼성전자는 여러 제품에 단계적으로 적용할 계획이며, AI 코딩 어시스턴트 ‘코드아이’를 포함한 여러 서비스를 제공할 예정입니다. \n\n이외에도 삼성전자는 다양한 기술 행사에 참가하며 AI 관련 최신 트렌드와 제품 개발을 지속하고 있습니다. 예를 들어, CES 2024와 AIMLA 2024 같은 글로벌 전시회와 컨퍼런스에 참가하여 AI, 5G, AR/VR 등 첨단 기술을 선보일 예정입니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="최근 삼성전자의 AI 동향은 2025년 기준으로 여러 혁신 기술과 전략이 진행되고 있음을 보여줍니다. 주요 내용은 다음과 같습니다:\n\n1. **AI 연산 성능 향상**: 최신 모바일 애플리케이션 프로세서인 '엑시노스 2400'은 NPU 성능을 기존보다 14.7배 향상시켜 AI 연산 능력을 크게 강화하고 있습니다.\n2. **반도체 및 메모리 기술 개발**: AI와 데이터 센터용 차세대 메모리 및 스토리지 기술, 예를 들어 CXL 기반 메모리 확장과 HBM4/4E, Z-NAND 